# Intro to Language Models

In this notebook we experiment with good ol' Markovian Language Models. 

While most contemporary use of LMs is neural, discussing old LMs will help us introduce some terminology, build some intuition on what (doesn't) work(s) and why and, more generally, start getting our hands dirty with language data within a very "interpretable" setting.

You are encouraged to play around with the code and modify / re-built parts of it as you fit: there is NO substitute for "tinkering with code" to understand how all the concepts fit together (corollary: all this code is written for pedagogical purposes, not for production use).

## Data loading

In [ ]:
# make sure you have the datasets library installed
# see: https://github.com/huggingface/datasets

# !pip install datasets

In [ ]:
import string

# some utils function
def get_finance_sentiment_dataset(split: str='sentences_allagree'):
    # load financial dataset from HF
    from datasets import load_dataset
    # https://huggingface.co/datasets/financial_phrasebank
    # by default, load just sentences for which all annotators agree
    dataset = load_dataset("financial_phrasebank", split)
    
    return dataset['train']


def get_finance_sentences():
    dataset = get_finance_sentiment_dataset()
    cleaned_dataset = [prepare_sentence(_['sentence']) for _ in dataset]
    # debug 
    print("{} cleaned sentences from finance dataset\n".format(len(cleaned_dataset)))
    
    return cleaned_dataset


def prepare_sentence(sentence: str):
    processed_sentence = pre_process_sentence(sentence)
    
    return tokenize_sentence(processed_sentence)


def pre_process_sentence(sentence: str):
    # this choices are VERY important. Here, we take a simplified 
    # view, remove the punctuations and just lower case everything
    lower_sentence = sentence.lower()
    # remove punctuation
    # nice suggestion from https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
    # if we change the exclude set, we can control what to exclude
    exclude = set(string.punctuation)
    return ''.join(ch for ch in lower_sentence if ch not in exclude)


def tokenize_sentence(sentence: str):
    # we use a vanilla tokenization technique (tokenize on white spaces): 
    # in production you may want to use a specialized
    # library to achieve the same goal, for example, the snippet below from https://spacy.io/api/tokenizer
    # shows how to tokenize an English sentence with Spacy
    
    # from spacy.lang.en import English
    # nlp = English()
    # Create a Tokenizer with the default settings for English
    # including punctuation rules and exceptions
    # tokenizer = nlp.tokenizer
    # tokens = tokenizer("This is a sentence")
    return sentence.split()


def get_corpus_from_text_file(text_file: str):
    # from a text file, we return a list of lists, where each list is a token in a sentence
    # ATTENTION: to get sentences we just split on new lines, remove empty lines and then split
    # on punctuation (;, .)
    # In a real setting, you would use specific libraries to detect sentence boundaries.
    with open(text_file, 'r') as file:
        sentences = [_ for _ in [s.strip() for s in file.read().replace(';', '.').split('.')] if _]
        # debug 
        print("{} raw sentences found in {}".format(len(sentences), text_file))
    
    # clean the sentences and remove empty ones
    cleaned_sentences = [_ for _ in [prepare_sentence(s) for s in sentences] if _]
    # debug 
    print("{} cleaned sentences from {}\n".format(len(cleaned_sentences), text_file))
            
    return cleaned_sentences

In [ ]:
# some test cases - get in the habit of test your functions ;-)
assert len(tokenize_sentence("This is my test sentence")) == 5
assert prepare_sentence("This is my sentence") == ["this", "is", "my", "sentence"]

In [ ]:
# we load different corpora here, so that we can switch between them later on
DATASETS = {}
# some shakespeare stuff, https://www.gutenberg.org/ebooks/author/65
DATASETS['william'] = get_corpus_from_text_file('shakespeare.txt')
# some Paul Graham stuff, http://www.paulgraham.com/articles.html
DATASETS['paul'] = get_corpus_from_text_file('graham.txt')
# some Finance stuff
DATASETS['finance'] = get_finance_sentences()

In [ ]:
from random import choice

In [ ]:
# debug by printing out some random sentences
for _ in range(5):
    print(choice(DATASETS['finance']), '\n')

## Zipf law in action

A recurrent problem in NLP is that a lot of what is said is relatively rare. According to Zipf Law (https://en.wikipedia.org/wiki/Zipf%27s_law), the most frequent word will occur approximately twice as often as the second most frequent word.

Let's do some counts and plot in our small datasets to get a feeling of what it means in practice...

In [ ]:
from collections import Counter

In [ ]:
def get_all_words_from_dataset(dataset: list):
    # we first FLAT a list of list 
    # [['i', 'am', 'jacopo'], ['you', 'are', 'funny']] ->
    # ['i', 'am', 'jacopo', 'you', 'are', 'funny']
    # and feed the list to a counter object
    return [word for sentence in dataset for word in sentence]

In [ ]:
test_set = [['i', 'am', 'jacopo'], ['you', 'are', 'funny']]
assert get_all_words_from_dataset(test_set) == ['i', 'am', 'jacopo', 'you', 'are', 'funny']

In [ ]:
counters = {}
for d, dataset in DATASETS.items():
    # get all words in a corpus
    all_words = get_all_words_from_dataset(dataset)
    counters[d] = Counter(all_words)
    # print out the most common words!
    print(d, counters[d].most_common(10), '\n')

In [ ]:
# bonus - we use the awesome package here: https://arxiv.org/pdf/1305.0215.pdf
# to do a proper powerlaw fit (paper is cool, if you want to see what is behind this)

# FROM https://github.com/jeffalstott/powerlaw

#!pip install powerlaw

In [ ]:
import powerlaw

In [ ]:
# we fit word frequency data
all_words = get_all_words_from_dataset(DATASETS['finance'])
c = Counter(all_words)
data = [_[1] for _ in c.most_common()]
print(data[:10])

In [ ]:
fit = powerlaw.Fit(data, discrete=True)
# compare the powerlaw fit and plot it
fit.distribution_compare('power_law', 'lognormal')
fig = fit.plot_ccdf(linewidth=3, label='Empirical Data')
fit.power_law.plot_ccdf(ax=fig, color='r', linestyle='--', label='Power law fit')
fit.lognormal.plot_ccdf(ax=fig, color='g', linestyle='--', label='Lognormal fit')
fig.set_ylabel(u"p(X≥x)")
fig.set_xlabel("Word Frequency")
handles, labels = fig.get_legend_handles_labels()
fig.legend(handles, labels, loc=3)
figname = 'FigLognormal'

## Vanilla Language Modelling

Thanks to the Markov assumption, we can use empirical frequencies to learn a language model for a given corpus. 

In [ ]:
# getting n-grams
def find_ngrams(tokens: list, n: int=2):
    # this is pretty cool: http://www.locallyoptimal.com/blog/2013/01/20/elegant-n-gram-generation-in-python/
    # try to understand what is going on here ;-)
    return zip(*[tokens[i:] for i in range(n)])

In [ ]:
print(list(find_ngrams(['this', 'is', 'NYC', 'and', 'I', 'love', 'it'], n=1)))
print(list(find_ngrams(['this', 'is', 'NYC', 'and', 'I', 'love', 'it'], n=2)))
print(list(find_ngrams(['this', 'is', 'NYC', 'and', 'I', 'love', 'it'], n=3)))

In [ ]:
assert len(list(find_ngrams(['this', 'is', 'NYC'], n=2))) == 2

In [ ]:
# nltk is another cool NLP package, which comes with some ngram and LM functionalities out of the box

#!pip install nltk

While it won't always be possible for us to do both a "from scracth" and a "package" implementation, Markov LM are a good test case (as in, the methods are straightforward and transparent enough to be re-created in a notebook).

In [ ]:
from nltk.util import ngrams

In [ ]:
print(list(ngrams(['this', 'is', 'NYC', 'and', 'I', 'love', 'it'], n=3)))

We introduce some custom tokens to indicate the start/end of a sentence:

In [ ]:
START_SYMBOL = 'FRE_7773_START'
STOP_SYMBOL = 'FRE_7773_STOP'

def pad_tokens(tokens: list, n=2):
    return [START_SYMBOL] * (n - 1) + tokens + [STOP_SYMBOL]

In [ ]:
print(pad_tokens(['I', 'love', 'it'], n=3))
assert pad_tokens(['this', 'is', 'NYC']) == [START_SYMBOL, 'this', 'is', 'NYC', STOP_SYMBOL]

### Training a vanilla language model

In [ ]:
def get_ngram_counter_for_lm(corpus: list, k: int):
    all_ngrams = []
    # loop over sentences, pad them with START and STOP symbol, and generate all ngram up until the chosen k
    for sentence in corpus:
        cnt_sentence = pad_tokens(sentence, n=k)
        for _ in range(1, k + 1): 
            all_ngrams = all_ngrams + list(find_ngrams(cnt_sentence, n=_))

    ngram_counter = Counter(all_ngrams)
    # debug
    print(ngram_counter.most_common(10))
    
    return ngram_counter

In [ ]:
# do a test run with a small corpus
cnt_corpus = [
    ['i', 'love', 'nyc'],
    ['i', 'teach', 'ml', 'in', 'nyc'],
    ['mike', 'lives', 'in', 'nyc'],
    ['mike', 'loves', 'nyc'],
    ['john', 'loves', 'chicago'],
    ['mike', 'is', 'a', 'great', 'teacher'],
    ['nyc', 'is', 'a', 'great', 'city'],
    ['chicago', 'is', 'a', 'big', 'city'],
    ['chicago', 'is', 'a', 'clean', 'city'],
    ['teaching', 'ml', 'is', 'great'],
    ['my', 'favorite', 'city', 'is', 'nyc']
]
n = 2
bigram_lm = get_ngram_counter_for_lm(corpus=cnt_corpus, k=n)

In [ ]:
from math import log

def calculate_sentence_probability(sentence: str, ngram_lm: Counter, n: int=2, verbose=False):
    tokens = prepare_sentence(sentence)
    cnt_sentence = pad_tokens(tokens, n=n)
    n_grams = list(find_ngrams(cnt_sentence, n=n))
    if verbose:
        print(cnt_sentence)
        print(n_grams)
    prob = 0.0
    for n_gram in n_grams:
        n_gram_count = ngram_lm[n_gram]
        n_minus_1_count = ngram_lm[n_gram[:-1]]
        n_gram_probability = n_gram_count / n_minus_1_count
        # debug
        if verbose:
            print(n_gram, n_gram[:-1], n_gram_count, n_minus_1_count)
        prob = prob + log(n_gram_probability)
    
    return prob

In [ ]:
test_sentences = [
    'mike loves chicago',
    'nyc is a big city',
    'nyc is a clean city'
]
for s in test_sentences:
    print(s, calculate_sentence_probability(s, bigram_lm, n), '\n')

_What happens when a test sentence features unseen ngrams?_

In [ ]:
# calculate_sentence_probability('teaching ml in chicago is great', bigram_lm, n)

In [ ]:
from collections import defaultdict

def build_probability_map(ngram_lm: Counter, n: int=2):
    # build a map storing the probability that a given n-gram follows a n-1 -gram
    n_gram_map = defaultdict(list)
    for (n_gram, count) in ngram_lm.most_common():
        # debug
        # print(n_gram, len(n_gram), count)
        if len(n_gram) == n:
            n_gram_map[n_gram[:-1]].append((n_gram, count))
    
    return n_gram_map

In [ ]:
n_gram_map = build_probability_map(bigram_lm, n=n)

In [ ]:
# print all the bigram starting with the START SYMBOL
n_gram_map[(START_SYMBOL,)]

In [ ]:
def generate_sentence(prompt: str, n_gram_map: dict, n: int=2, is_random=False):
    tokens = prepare_sentence(prompt)
    # remove the STOP symbol at the right
    sentence = pad_tokens(tokens, n=n)[:-1]
    while STOP_SYMBOL not in sentence or len(sentence) == 20:
        n_gram_key = tuple(sentence[len(sentence) - (n - 1):])
        # debug
        # print(sentence, n_gram_key)
        # possible continuations
        continuations = n_gram_map[n_gram_key]
        # pick the first one, or a random one
        new_token = continuations[0][0][n - 1:] if not is_random else choice(continuations)[0][n - 1:]
        sentence = sentence + list(new_token)
                              
    return ' '.join(sentence)

In [ ]:
# generate a sentence
generate_sentence('mike', n_gram_map, n, is_random=False)

In [ ]:
# generate a sentence
generate_sentence('mike', n_gram_map, n, is_random=True)

In [ ]:
generate_sentence('', n_gram_map, n, is_random=True)

In [ ]:
# now run it on one of our dataset
cnt_corpus = DATASETS['william']
n = 2 
bigram_lm = get_ngram_counter_for_lm(corpus=cnt_corpus, k=n)
n_gram_map = build_probability_map(bigram_lm, n=n)

In [ ]:
# print all the bigram starting with the START SYMBOL
n_gram_map[(START_SYMBOL,)][:10]

In [ ]:
# generate a sentence
generate_sentence('what', n_gram_map, n)

_A trigram model has the same underlying logic..._

In [ ]:
cnt_corpus = DATASETS['finance']
n = 3
trigram_lm = get_ngram_counter_for_lm(corpus=cnt_corpus, k=n)
n_gram_map = build_probability_map(trigram_lm, n=n)
n_gram_map[(START_SYMBOL, 'the')][:10]

In [ ]:
# generate a sentence
generate_sentence('the new', n_gram_map, n, is_random=True)

## Training a LM with NLTK

Now that we understand how a simple n-gram model works, we can use some abstraction to train and test it faster (the following code uses NLTK API -> https://www.nltk.org/api/nltk.lm.html)

In [ ]:
from nltk.lm.preprocessing import padded_everygram_pipeline
cnt_corpus = DATASETS['finance']
n = 3
train_data, padded_sents = padded_everygram_pipeline(3, cnt_corpus)
# what does the new train_data vaiable hold? It is simply a collection of n-grams up to n for the sentences
# in the corpus
print(list(list(train_data)[0])[:10])

In [ ]:
from nltk.lm import MLE, Laplace

def create_nltk_model(corpus: list, n: int):
    train_data, padded_sents = padded_everygram_pipeline(n, corpus)
    ngram_model = Laplace(n)
    ngram_model.fit(train_data, padded_sents)
    # debug - print the vocabulary
    print(len(ngram_model.vocab))
    
    return ngram_model

# we now fit the model to the data
cnt_corpus = DATASETS['paul']
n = 3
_model = create_nltk_model(cnt_corpus, n)

In [ ]:
def generate_sentence_nltk(model, num_words):
    content = []
    for token in model.generate(num_words):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
        
    return ' '.join(content)

In [ ]:
for _ in range(5):
    print(generate_sentence_nltk(_model, 20), '\n')

## Evaluate a Language Model

NTLK comes with an out of the box method to calculate perplexity - we can see it in action here on a sample training and test set

In [ ]:
text = [['a', 'b'], ['a', 'b', 'c'], ['a', 'c', 'd', 'c', 'e', 'f']]
train, vocab = padded_everygram_pipeline(2, text)
lm = MLE(2)
lm.fit(train, vocab)
# print(lm.score("b", ["a"]))
# print(lm.logscore("a"))
# print(lm.logscore("b", ["a"]))

In [ ]:
from statistics import mean

def calculate_perplexity(log_probs: list):
    return 2** (-1 * mean(log_probs))

In [ ]:
test = [('a', 'b'), ('c', 'd')]
log_probs = [lm.logscore(t[-1], t[:-1]) for t in test]

# calculate perplexity from scratch and with the built-in function, as a double check!
print(lm.perplexity(test))
print(calculate_perplexity(log_probs))

We now turn a more "realistic" LM, splitting one of our dataset in train and test set and measuring perplexity...

In [ ]:
from sklearn.model_selection import train_test_split

train_corpus, test_corpus = train_test_split(DATASETS['paul'], test_size=0.1)
print(len(train_corpus), len(test_corpus))
test_corpus[0][:10]

In [ ]:
n = 2
_model = create_nltk_model(train_corpus, n)
_model.perplexity(test_corpus)

Based on our domain knowledge, we could also evaluate a model through some "behavioral checks" (e.g. https://arxiv.org/abs/2005.04118), that is, we can prepare input-output pairs and make sure the model behave as expected in edge cases...


In [ ]:
def paul_behavioral_checks(nltk_model):
    # write some behavioral tests based on paul graham known themes...
    assert nltk_model.logscore("startup") > nltk_model.logscore("politics")
    assert nltk_model.counts["nerds"] > 0
    assert nltk_model.logscore("founder", ["startup"]) > nltk_model.logscore("founder", ["italian"])
    
    print("All checks passed!")
    return 


paul_behavioral_checks(_model)

## Applied LM: How to Write a Spelling Corrector (Norvig's Post)

Language models are important components of many NLP application. A pedagogically extraordinary post by Peter Norvig (https://norvig.com/spell-correct.html) shows how to implement a noisy channel model in Python for typo correction.

The code below is taken directly from the original post (with some minor modifications to work with our corpora) and it is reported here for convenience: one of your homework assignment asks you to improve upon his original work by either improving the language model or the error model.

In [ ]:
import re

# pick a corpus
cnt_corpus = DATASETS['paul']
# get word frequencies
WORDS = Counter(get_all_words_from_dataset(cnt_corpus))
WORDS.most_common(10)

In [ ]:
def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
def unit_tests():
    assert correction('beause') == 'because'                # insert
    assert correction('srartupz') == 'startup'              # replace 2
    assert correction('bycycle') == 'bicycle'               # replace
    assert correction('inconvient') == 'inconvenient'       # insert 2
    assert correction('arrainged') == 'arranged'            # delete
    assert correction('peotry') =='poetry'                  # transpose
    assert correction('peotryy') =='poetry'                 # transpose + delete
    assert correction('word') == 'word'                     # known
    assert correction('quintessential') == 'quintessential' # unknown
    assert WORDS.most_common(1)[0][0] == 'the'
    assert P('trafalgar') == 0

    return 'unit_tests pass'


def spelltest(tests, verbose=False):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.time()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in WORDS)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'.format(wrong, w, WORDS[w], right, WORDS[right]))
    dt = time.time() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))
    
    return


def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

print(unit_tests())
spelltest(Testset(open('spell-testset1.txt'))) # Development set
spelltest(Testset(open('spell-testset2.txt'))) # Final test set